In [1]:
! pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xvf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [4]:
print (os.listdir('./sample_data'))

['anscombe.json', 'README.md', 'california_housing_train.csv', 'california_housing_test.csv', 'mnist_train_small.csv', 'mnist_test.csv']


In [5]:
file_loc = './sample_data/california_housing_train.csv'

In [6]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
#spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
# = spark.read.csv(file_loc, inferSchema=True, header =True)

In [9]:
#print (type(df))

In [10]:
#df.printSchema()

In [11]:
#df.show()

In [12]:
from pyspark import SparkContext

In [13]:
sc = SparkContext()

In [16]:
%%writefile Example.txt
First line
Second line
Third line
Fourth line

Writing Example.txt


In [17]:
textFile = sc.textFile('Example.txt')

In [18]:
textFile.count()

4

In [19]:
textFile.first()

'First line'

In [21]:
first = textFile.filter(lambda line: 'First' in line)

In [22]:
first

PythonRDD[5] at RDD at PythonRDD.scala:53

In [23]:
first.collect()

['First line']

In [24]:
%%writefile Example2.txt
first
second line
the third line
then a fourth line

Writing Example2.txt


In [26]:
"""from pyspark import SparkContext

sc = SparkContext()"""

'from pyspark import SparkContext\n\nsc = SparkContext()'

In [28]:
text = sc.textFile('Example2.txt')

In [30]:
words = text.map(lambda line: line.split())

In [31]:
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line']]

In [32]:
text.collect()

['first', 'second line', 'the third line', 'then a fourth line']

In [35]:
text.flatMap(lambda line: line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line']

In [36]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [37]:
services = sc.textFile('services.txt')

In [38]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [40]:
services.map(lambda line: line.split()).collect()

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [44]:
clean = services.map(lambda line: line[1:] if line[0] == '#' else line)

clean = clean.map(lambda line:line.split())

clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [47]:
# Getting State and Amount
pairs = clean.map(lambda list: (list[3], list[-1]))
pairs.collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [49]:
# Using reduceByKey to obtain groupby
rekey = pairs.reduceByKey(lambda amt1, amt2 : float(amt1) + float(amt2))
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [52]:
# Removing State and Amount title
values = rekey.filter(lambda x: not x[0] == 'State')
# Sort values by Amount
sorted = values.sortBy(lambda stAmount: stAmount[1], ascending = False) # sortBy second item

sorted.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [53]:
x = ['ID', 'State', 'Amount']

In [56]:
def func(lst):
  return lst[-1]

In [57]:
def func2(id_st_amt):
  #Tuple Unpacking
  (id, st, amt) = id_st_amt
  return amt

In [59]:
func2(x), func(x)

('Amount', 'Amount')